# The ScholoApp - Exploratory Data Analysis

## Introduction

In [1]:
from pyspark.sql import SparkSession
import pandas as pd

In [2]:
spark = SparkSession.builder.\
        appName("eda_ScholoApp").\
        master("spark://spark-master:7077").\
        getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/06/12 16:28:02 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
/spark/python/pyspark/context.py:317: FutureWarning: Python 3.7 support is deprecated in Spark 3.4.
  warnings.warn("Python 3.7 support is deprecated in Spark 3.4.", FutureWarning)


In [3]:
spark

## Create / Access the database

In [42]:
import pymongo
from pymongo import MongoClient

## Note that 'mymongo' is the name of the docker container
client = MongoClient('mongodb://mymongo:27017/')

Recall that the mongo container is connected to a docker volume.

The list of database should be the three defaults, admin, config, and local plus the one for the ScholoApp (unless it is the first run, with only the defaults present)

In [49]:
for i in client.list_databases():
    print(i)

{'name': 'admin', 'sizeOnDisk': 40960, 'empty': False}
{'name': 'config', 'sizeOnDisk': 61440, 'empty': False}
{'name': 'local', 'sizeOnDisk': 73728, 'empty': False}


### The Users

In [4]:
## it is a good idea to declare the schema up-front to avoid computation and catch errors

In [32]:
## The file, the raw csv, will be downloaded disk. 

import wget

url = "https://raw.githubusercontent.com/andreas-masaoutis/ScholoApp/main/data/raw_data/ScholoApp_users.csv"

wget.download(url)

100% [............................................................................] 187686 / 187686

'ScholoApp_users.csv'

In [33]:
import csv

with open("ScholoApp_users.csv") as f:
    reader = csv.reader(f)
    for row in reader:
        print(row)

['user_id', 'account_creation_time', 'seniority']
['8be2f87809280cfd85e48', '2022-06-05 10:06:17.505', 'Freshman']
['9ffa8ba809bdfad8500f4', '2022-06-12 12:41:27.704', 'Sophomore']
['8228b2280a4e57bb40788', '2022-06-19 08:56:37.229', 'Sophomore']
['88f251f80921447bea2a4', '2022-06-05 02:23:13.909', 'Sophomore']
['9ceef2380a3289d1ff78a', '2022-06-18 01:09:59.453', 'Sophomore']
['8aeb5c980ad08b911f135', '2022-06-25 12:56:35.441', 'Sophomore']
['8b5295c809ea182c82148', '2022-06-14 14:53:00.398', 'Sophomore']
['8fad4ae80bb8ee478b0e7', '2022-07-06 13:29:18.591', 'Junior']
['8b75b80809f3d87c82148', '2022-06-15 01:58:09.276', 'Freshman']
['87e8ba980b488e5c7002f', '2022-07-01 05:29:19.244', 'Sophomore']
['8cc001c80959e8dd85e48', '2022-06-07 18:49:27.193', 'Senior']
['8e13d0b80bac518dd93f9', '2022-07-05 23:08:18.732', 'Freshman']
['8a875958097f893bea2a4', '2022-06-09 13:30:05.040', 'Freshman']
['8e3847580b7930878b0e7', '2022-07-03 12:57:55.023', 'Sophomore']
['9cd484c80a2cc451ff78a', '2022-06-1

In [34]:
users_pandasDF = pd.read_csv("ScholoApp_users.csv")

In [39]:
## We do not need the file anymore.
## The following is a valid BASH command - it might not work on WINDOWS
! rm "ScholoApp_users.csv"

rm: cannot remove ‘ScholoApp_users.csv’: No such file or directory


In [36]:
users_sparkDF = spark.createDataFrame(users_pandasDF)

In [37]:
users_sparkDF.show()

+--------------------+---------------------+---------+
|             user_id|account_creation_time|seniority|
+--------------------+---------------------+---------+
|8be2f87809280cfd8...| 2022-06-05 10:06:...| Freshman|
|9ffa8ba809bdfad85...| 2022-06-12 12:41:...|Sophomore|
|8228b2280a4e57bb4...| 2022-06-19 08:56:...|Sophomore|
|88f251f80921447be...| 2022-06-05 02:23:...|Sophomore|
|9ceef2380a3289d1f...| 2022-06-18 01:09:...|Sophomore|
|8aeb5c980ad08b911...| 2022-06-25 12:56:...|Sophomore|
|8b5295c809ea182c8...| 2022-06-14 14:53:...|Sophomore|
|8fad4ae80bb8ee478...| 2022-07-06 13:29:...|   Junior|
|8b75b80809f3d87c8...| 2022-06-15 01:58:...| Freshman|
|87e8ba980b488e5c7...| 2022-07-01 05:29:...|Sophomore|
|8cc001c80959e8dd8...| 2022-06-07 18:49:...|   Senior|
|8e13d0b80bac518dd...| 2022-07-05 23:08:...| Freshman|
|8a875958097f893be...| 2022-06-09 13:30:...| Freshman|
|8e3847580b7930878...| 2022-07-03 12:57:...|Sophomore|
|9cd484c80a2cc451f...| 2022-06-17 18:44:...| Freshman|
|858170880

In [38]:
users_sparkDF.head(5)

[Row(user_id='8be2f87809280cfd85e48', account_creation_time='2022-06-05 10:06:17.505', seniority='Freshman'),
 Row(user_id='9ffa8ba809bdfad8500f4', account_creation_time='2022-06-12 12:41:27.704', seniority='Sophomore'),
 Row(user_id='8228b2280a4e57bb40788', account_creation_time='2022-06-19 08:56:37.229', seniority='Sophomore'),
 Row(user_id='88f251f80921447bea2a4', account_creation_time='2022-06-05 02:23:13.909', seniority='Sophomore'),
 Row(user_id='9ceef2380a3289d1ff78a', account_creation_time='2022-06-18 01:09:59.453', seniority='Sophomore')]

In [6]:
users.show(10)

+--------------------+---------------------+---------+
|             user_id|account_creation_time|seniority|
+--------------------+---------------------+---------+
|8be2f87809280cfd8...| 2022-06-05 10:06:...| Freshman|
|9ffa8ba809bdfad85...| 2022-06-12 12:41:...|Sophomore|
|8228b2280a4e57bb4...| 2022-06-19 08:56:...|Sophomore|
|88f251f80921447be...| 2022-06-05 02:23:...|Sophomore|
|9ceef2380a3289d1f...| 2022-06-18 01:09:...|Sophomore|
|8aeb5c980ad08b911...| 2022-06-25 12:56:...|Sophomore|
|8b5295c809ea182c8...| 2022-06-14 14:53:...|Sophomore|
|8fad4ae80bb8ee478...| 2022-07-06 13:29:...|   Junior|
|8b75b80809f3d87c8...| 2022-06-15 01:58:...| Freshman|
|87e8ba980b488e5c7...| 2022-07-01 05:29:...|Sophomore|
+--------------------+---------------------+---------+
only showing top 10 rows



In [7]:
users.count()

3444

In [8]:
type( users.first() )

pyspark.sql.types.Row

In [9]:
users.first()

Row(user_id='8be2f87809280cfd85e48', account_creation_time=datetime.datetime(2022, 6, 5, 10, 6, 17, 505000), seniority='Freshman')

In [10]:
users.cache()

DataFrame[user_id: string, account_creation_time: timestamp, seniority: string]

In [11]:
users.groupby('seniority').count().orderBy('count', ascending=False).show()

+---------+-----+
|seniority|count|
+---------+-----+
|Sophomore| 1250|
|   Junior|  766|
|   Senior|  746|
| Freshman|  682|
+---------+-----+



### The actions

In [12]:
actions = spark.read.option("header", "true").option("inferSchema", "true").csv("../raw_data/ScholoApp_actions.csv")
actions.printSchema()

root
 |-- user_id: string (nullable = true)
 |-- action_time: timestamp (nullable = true)
 |-- user_action: string (nullable = true)



In [13]:
actions.show(10)

+--------------------+-------------------+-------------------+
|             user_id|        action_time|        user_action|
+--------------------+-------------------+-------------------+
|998552f809bd21eb1...|2022-06-12 13:42:18|Browse magic spells|
|82fb8a480a7498cb4...|2022-06-22 07:37:31|Browse magic spells|
|881009180aa42310d...|2022-06-23 12:55:18|Browse magic spells|
|81a52da80a383d3b4...|2022-06-19 20:14:24|Browse magic spells|
|8b00c28809a20d5be...|2022-07-04 07:28:33|Browse magic spells|
|825c9c980b75fa98a...|2022-07-04 09:53:16|Browse magic spells|
|8b4c17180ae11a811...|2022-06-26 09:58:11|Browse magic spells|
|8f9a85880bb831e78...|2022-07-06 13:48:28|Browse magic spells|
|82dd39880a8dd38b4...|2022-09-06 15:27:25|Browse magic spells|
|8a988d28098c4a3be...|2022-06-10 11:28:40|Browse magic spells|
+--------------------+-------------------+-------------------+
only showing top 10 rows



In [29]:
actions.count()

171677

In [14]:
actions.cache()

DataFrame[user_id: string, action_time: timestamp, user_action: string]

In [15]:
from pyspark.sql.functions import avg, stddev, stddev_pop

actions.groupby('user_id').count().agg(avg('count'), stddev('count'), stddev_pop('count')).show()

+------------------+------------------+------------------+
|        avg(count)|stddev_samp(count)| stddev_pop(count)|
+------------------+------------------+------------------+
|51.155244338498214|136.86470754796756|136.84431498172938|
+------------------+------------------+------------------+



In [16]:
actions.groupby('user_id').count().describe().show()

+-------+--------------------+------------------+
|summary|             user_id|             count|
+-------+--------------------+------------------+
|  count|                3356|              3356|
|   mean|                null|51.155244338498214|
| stddev|                null|136.86470754796756|
|    min|73309ce801c2dd884...|                 1|
|    max|9fff8cb80ac85f81f...|              3048|
+-------+--------------------+------------------+



In [21]:
actions.createOrReplaceTempView('actions_table')

In [22]:
users.createOrReplaceTempView('users_table')

In [109]:
combined_df = spark.sql(
    """
    SELECT
        *
    FROM
        actions_table
    RIGHT JOIN
        users_table
        ON
        actions_table.user_id = users_table.user_id
    """
    ).cache()

In [110]:
combined_df.count()

171765

In [111]:
combined_df.select('actions_table.user_id').distinct().count()

3357

In [112]:
combined_df.select('users_table.user_id').distinct().count()

3444

In [125]:
combined_df.filter(combined_df['actions_table.user_id'].isNull()).count()

88

In [127]:
combined_df.filter(combined_df['actions_table.user_id'].isNull()).distinct().show()

+-------+-----------+-----------+--------------------+---------------------+---------+
|user_id|action_time|user_action|             user_id|account_creation_time|seniority|
+-------+-----------+-----------+--------------------+---------------------+---------+
|   null|       null|       null|8efa2a280b9758578...| 2022-07-04 23:15:...|Sophomore|
|   null|       null|       null|8943e83809335c5be...| 2022-06-05 22:58:...|Sophomore|
|   null|       null|       null|8fb4a4580bb98e578...| 2022-07-06 14:11:...|Sophomore|
|   null|       null|       null|8cd594b80b785abdd...| 2022-07-03 12:00:...|   Junior|
|   null|       null|       null|8f2155e80b9e79b78...| 2022-07-05 07:23:...|   Junior|
|   null|       null|       null|8cdcd4480b793efdd...| 2022-07-03 13:01:...|   Senior|
|   null|       null|       null|8e4539280b7b03b78...| 2022-07-03 15:02:...|Sophomore|
|   null|       null|       null|8f0ad3580b9aa1d78...| 2022-07-05 03:00:...| Freshman|
|   null|       null|       null|8e7834e80b